In [1]:
import itertools
import math
import numpy as np

In [2]:
np.random.uniform(3, 10, 10).astype(float)

array([7.79021685, 5.45161907, 7.29159991, 4.51546389, 3.30651251,
       6.4368665 , 4.06641568, 3.35824923, 3.2123195 , 8.68454206])

In [3]:
cat = ['A', 'B', 'C']

In [4]:
np.random.choice(cat, 10)

array(['B', 'B', 'B', 'C', 'B', 'B', 'C', 'C', 'A', 'C'], dtype='<U1')

In [5]:
vec = ['i', 'j', 'k', 'l', 'm', 'n']

In [6]:
vec = [1, 2, 3, 4, 5]

In [7]:
import time
n = 10000

In [8]:
t0s = time.time()
i = np.array([np.random.choice(vec, 3, False) for _ in range(n)])
t0 = time.time() - t0s
print(t0)

0.2502319812774658


In [9]:
np.random.permutation(vec)

array([5, 3, 4, 2, 1])

In [87]:
tuple(np.random.choice(vec, (10, 4)))

(array([1, 5, 3, 1]),
 array([1, 3, 4, 4]),
 array([1, 1, 4, 4]),
 array([2, 1, 2, 3]),
 array([2, 3, 4, 2]),
 array([1, 5, 5, 5]),
 array([4, 4, 5, 3]),
 array([3, 1, 4, 5]),
 array([1, 1, 3, 2]),
 array([4, 4, 4, 4]))